Import packages

In [2]:
python -m spacy download da_core_news_sm

SyntaxError: invalid syntax (3771486239.py, line 1)

In [38]:
from gensim.models.wrappers import LdaMallet

ModuleNotFoundError: No module named 'gensim.models.wrappers'

In [37]:
# nltk stuff
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# basic packages
import pandas as pd
import numpy as np
from pprint import pprint
import re
import gc
# gensim and related
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.wrappers import LdaMallet

# spacy
import spacy
spacy.load('da_core_news_sm')

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /home/ucloud/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'gensim.models.wrappers'

In [ ]:
# Initialize spacy 'da' model, keeping only tagger component (for efficiency)
nlp = spacy.load('da_core_news_sm', disable=['parser', 'ner'])

# LDA

Prepare stopwords

In [ ]:
# NLTK Stop words
stop_words = stopwords.words('danish') # Tror stopwords fra spacy modellen er bedre - det er i hvert fald en længere liste
stop_words = list(nlp.Defaults.stop_words) # stopwords fra spacy model
stop_words.extend(['originalartiklen', 'originalartikel', 'ea670633', 'Originalartiklen', 'feature', 'øh', 'øhm', 'sådan'])

Importing data

In [3]:
# Loading data
mepsda_df = pd.read_csv('/work/Ccp-MePSDA/output/collected_data/mepsda_df.csv')
mepsda_df.drop(columns='index', inplace=True)
mepsda_df['text'] = mepsda_df['text'].astype(str)

# aggregating each text under the same title
mepsda_df = mepsda_df.groupby(['title', 'source']).agg({'text':'\n'.join})

In [4]:
# cleaning

# Converting to list
data = mepsda_df.text.values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_2290/1964869558.py:7: SyntaxWarning: invalid escape sequence '\s'
  data = [re.sub('\s+', ' ', sent) for sent in data]


Tokenize words and clean-up

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['feature', 'okt', 'opdateret', 'okt', 'usunde', 'skønhedsidealer', 'og', 'proppet', 'med', 'sex', 'foto', 'nimbus', 'film', 'jonas', 'risvig', 'er', 'flankeret', 'af', 'sine', 'to', 'kontra', 'andrea', 'hjejle', 'og', 'anna', 'lehrmann', 'som', 'ifølge', 'nogle', 'unge', 'cementerer', 'et', 'undertrykkende', 'kvindeideal', 'instruktørens', 'værker', 'hvorfor', 'skal', 'pigerne', 'altid', 'være', 'smukke', 'tynde', 'rige', 'og', 'sex', 'fikserede', 'jonas', 'risvig', 'har', 'trukket', 'stikket', 'efter', 'shitstorm', 'men', 'konsulent', 'tager', 'instruktøren', 'forsvar', 'af', 'frederik', 'hoff', 'og', 'claus', 'christensen', 'arige', 'jonas', 'risvig', 'har', 'med', 'sine', 'mange', 'ungdomsserier', 'og', 'film', 'længe', 'været', 'anmeldernes', 'darling', 'et', 'af', 'sine', 'første', 'interviews', 'fortalte', 'han', 'til', 'ekko', 'at', 'han', 'opgav', 'at', 'søge', 'ind', 'pa', 'den', 'danske', 'filmskole', 'fordi', 'lederen', 'af', 'sagde', 'at', 'man', 'skulle', 'have', 'haft',

Creating bigram and trigram models

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=75) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=75)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['feature', 'okt', 'opdateret', 'okt', 'usunde', 'skønhedsidealer', 'og', 'proppet', 'med', 'sex', 'foto', 'nimbus', 'film', 'jonas_risvig', 'er', 'flankeret', 'af', 'sine', 'to', 'kontra', 'andrea_hjejle', 'og', 'anna_lehrmann', 'som', 'ifølge', 'nogle', 'unge', 'cementerer', 'et', 'undertrykkende', 'kvindeideal', 'instruktørens', 'værker', 'hvorfor', 'skal', 'pigerne', 'altid', 'være', 'smukke', 'tynde', 'rige', 'og', 'sex', 'fikserede', 'jonas_risvig', 'har', 'trukket', 'stikket', 'efter', 'shitstorm', 'men', 'konsulent', 'tager', 'instruktøren', 'forsvar', 'af', 'frederik', 'hoff', 'og', 'claus', 'christensen', 'arige_jonas_risvig', 'har', 'med', 'sine', 'mange', 'ungdomsserier', 'og', 'film', 'længe', 'været', 'anmeldernes', 'darling', 'et', 'af', 'sine', 'første', 'interviews', 'fortalte', 'han', 'til', 'ekko', 'at', 'han', 'opgav', 'at', 'søge', 'ind', 'pa', 'den', 'danske_filmskole', 'fordi', 'lederen', 'af', 'sagde', 'at', 'man', 'skulle', 'have', 'haft', 'en', 'darlig', 'barn

Remove stopwords, make bigrams and lemmatize

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Calling above functions

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'da' model, keeping only tagger component (for efficiency)
nlp = spacy.load('da_core_news_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['okt', 'opdatere', 'okt', 'usund', 'skønhedsideale', 'propp', 'sex', 'foto', 'film', 'flanker', 'kontra', 'ung', 'cementere', 'undertrykke', 'kvindeideal', 'instruktør', 'værk', 'hvorfor', 'pig', 'altid', 'smuk', 'tynd', 'rig', 'sex', 'fiksere', 'trække', 'stikke', 'shitstorm', 'konsulent', 'tage', 'instruktør', 'forsvar', 'christens', 'arig', 'jonas_risvig', 'ungdomsserie', 'film', 'længe', 'anmeld', 'darling', 'første', 'interviews', 'fortælle', 'ekko', 'opgive', 'søge', 'pa', 'danske_filmskole', 'leder', 'sige', 'have', 'darlig', 'barndom', 'kunstner', 'annonce', 'sidde', 'ung', 'dreng', 'opvokset', 'pa', 'stille', 'villavej', 'føle', 'provinsbaggrund', 'betyde', 'lav', 'film', 'sige', 'sted', 'give', 'gå', 'egen', 'autodidakte', 'vej', 'musikvideo', 'kortfilm', 'serie', 'mange', 'kort', 'produktion', 'se', 'pa', 'ekko_shortlist', 'vinde', 'publikumspris', 'ekko_shortlist', 'awards', 'begynde', 'inddrag', 'ung', 'tilblivels', 'serie', 'lave', 'sagar', 'egen', 'filmskole', 'barndom

creating dictionary and corpus

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [10]:
# Check document length distribution
doc_lengths = [len(doc) for doc in data]
print(f"Minimum document length: {min(doc_lengths)}")
print(f"Average document length: {sum(doc_lengths) / len(doc_lengths)}")

Minimum document length: 92
Average document length: 28775.43307086614


Building Topic Model

In [16]:
gc.collect()

0

In [32]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=22,  
    random_state=666,
    update_every=0,  # Update only after each pass through the corpus
    iterations=100,  
    chunksize=350,  
    passes=10,
    alpha='auto',
    eta='auto',
    decay=0.5, 
    per_word_topics=True
)

In [33]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(10,
  '0.000*"pa" + 0.000*"instruktør" + 0.000*"skuespiller" + 0.000*"sige" + '
  '0.000*"ung" + 0.000*"producer" + 0.000*"holde" + 0.000*"se" + '
  '0.000*"situation" + 0.000*"altsa"'),
 (11,
  '0.000*"instruktør" + 0.000*"sige" + 0.000*"pa" + 0.000*"skuespiller" + '
  '0.000*"skrive" + 0.000*"gøre" + 0.000*"ogsa" + 0.000*"altsa" + '
  '0.000*"producer" + 0.000*"sadan"'),
 (14,
  '0.025*"dreng" + 0.012*"dag" + 0.012*"sige" + 0.010*"pa" + 0.009*"mand" + '
  '0.009*"tale" + 0.009*"far" + 0.009*"svært" + 0.008*"jonas_risvig" + '
  '0.006*"hvordan"'),
 (19,
  '0.000*"pa" + 0.000*"sige" + 0.000*"lidt" + 0.000*"lige" + 0.000*"sadan" + '
  '0.000*"altsa" + 0.000*"komme" + 0.000*"god" + 0.000*"tro" + 0.000*"bare"'),
 (17,
  '0.001*"sadan" + 0.000*"pa" + 0.000*"altsa" + 0.000*"lige" + 0.000*"sidde" + '
  '0.000*"bare" + 0.000*"ogsa" + 0.000*"sige" + 0.000*"lidt" + 0.000*"lave"'),
 (18,
  '0.009*"tv" + 0.008*"tillid" + 0.007*"orlov" + 0.005*"genoprette" + '
  '0.005*"jon_stephense" + 0.004*"f

In [34]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.143730783995435

Coherence Score:  0.30416357488442586


In [35]:
# Check if the corpus has enough data
for idx, bow in enumerate(corpus[:5]):  # Check the first few documents
    print(f"Document {idx}: {bow}")  # Each document should have multiple tuples representing word IDs and counts

Document 0: [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 4), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 3), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 4), (25, 2), (26, 2), (27, 1), (28, 1), (29, 2), (30, 1), (31, 7), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 4), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 3), (70, 1), (71, 1), (72, 2), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 2), (80, 2), (81, 5), (82, 1), (83, 2), (84, 1), (85, 1), (86, 2), (87, 2), (88, 1), (89, 1), (90, 2), (91, 1), (92, 3), (93, 1), (94, 2), (95, 2), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 4), (103, 1), (104, 4), (105, 2), (106, 1), (107, 2), (108, 1), (109, 1

In [30]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.259234 -0.079213       1        1  83.624275
20     0.234642  0.041923       2        1   4.654949
12     0.232815 -0.068291       3        1   2.959233
5      0.238288 -0.102456       4        1   2.581760
21     0.208125  0.119573       5        1   2.235211
16     0.114099  0.151471       6        1   1.581960
2      0.096445  0.060176       7        1   0.711099
7      0.074011 -0.067064       8        1   0.417375
22     0.012973  0.031597       9        1   0.412063
6     -0.033811  0.034573      10        1   0.244648
8     -0.061431  0.013351      11        1   0.117126
13    -0.011218 -0.023919      12        1   0.116407
18    -0.104674  0.007941      13        1   0.112703
3     -0.110114  0.009881      14        1   0.090667
19    -0.140332 -0.001302      15        1   0.045434
1     -0.086173  0.000061      16        1   0.041025
0     -0.068312 -0.036735      17        1   0.030900
9     -0.130492 -0.012366      18        1   0.017610
15    -0.054759 -0.036045      19        1   0.001036
23    -0.090270 -0.023487      20        1   0.001018
17    -0.135924 -0.009961      21        1   0.000970
14    -0.142042  0.002679      22        1   0.000875
11    -0.149217 -0.006382      23        1   0.000858
10    -0.151865 -0.006005      24        1   0.000797, topic_info=         Term         Freq        Total Category  logprob  loglift
385        pa  5635.000000  5635.000000  Default  30.0000  30.0000
441     sadan  7102.000000  7102.000000  Default  29.0000  29.0000
31       bare  3894.000000  3894.000000  Default  28.0000  28.0000
469      sige  3516.000000  3516.000000  Default  27.0000  27.0000
619     altsa  2946.000000  2946.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
203      gøre     0.000128  2670.937615  Topic24  -9.6634  -5.1121
80        dag     0.000127  1512.169932  Topic24  -9.6727  -4.5525
338  menneske     0.000127   715.712407  Topic24  -9.6767  -3.8085
314        ma     0.000126   649.792395  Topic24  -9.6792  -3.7144
537       tid     0.000126   816.854533  Topic24  -9.6805  -3.9445

[1658 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2808      8  1.130103        Jamie
5700      1  0.628847    København
2404      4  0.104378  Skuespiller
2404      6  0.835021  Skuespiller
2231      5  0.953892  afleveringe
...     ...       ...          ...
611       3  0.012795          øje
611       4  0.051180          øje
611       5  0.063975          øje
611       7  0.051180          øje
3294      6  0.981875      ølkasse

[2449 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 21, 13, 6, 22, 17, 3, 8, 23, 7, 9, 14, 19, 4, 20, 2, 1, 10, 16, 24, 18, 15, 12, 11])

In [36]:
mallet_path = '/work/Ccp-MePSDA/modules/mallet-2.0.8' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

AttributeError: module 'gensim.models' has no attribute 'wrappers'

# LDA2Vec